## Configuração (Somente quando necessário)

In [ ]:
! pip install pymysql;

## Restaurar danco de dados

In [1]:
import pymysql
import os
import time
from datetime import datetime
import subprocess
import ctypes

def listar_databases_atuais(cursor):
    cursor.execute('SHOW DATABASES')
    bancos_sommusgestor = [db[0] for db in cursor.fetchall() if 'sommusgestor' in db[0]]
    return bancos_sommusgestor

def deletar_database(cursor, nome):
    cursor.execute(f"DROP DATABASE {nome}")

def criar_database(cursor, nome):
    if(nome.startswith("sommusgestor") and not nome.endswith("_old")):
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {nome}")

def restaurar(banco, host, porta, usuario, password, diretorio):
    if(banco.startswith("sommusgestor") and not banco.endswith("_old")):
        comando = f'mysql -h {host} -P {porta} -u {usuario} -p{password} {banco} < {diretorio}/{banco}.sql'
        processo = subprocess.run(comando, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if processo.returncode == 0:
            print(f"   ⚪ Banco {banco}")
        else:
            raise Exception(f'Banco [{banco}] \n{processo.stderr.decode()}')

# Constantes para os tipos de botões
MB_YESNO = 4
IDYES = 6
IDNO = 7

# Função para exibir uma caixa de diálogo do Windows
def exibir_caixa_mensagem(titulo, mensagem):
    resposta = ctypes.windll.user32.MessageBoxW(0, mensagem, titulo, MB_YESNO)
    return resposta == IDYES

class ProcessoCanceladoException(Exception):
    pass

In [2]:
HOST = 'localhost'
PORTA = 3307
USUARIO = 'root'
PASSWORD = 'root'
PASTA = 'backups'
SUBPASTA = 'master_playlist_2023-12-29_08-58-49'
DIRETORIO_BACKUP = f'{PASTA}/{SUBPASTA}'

if __name__ == '__main__':
    try:
        seguir = exibir_caixa_mensagem("Confirmação", f"Deseja restaurar backup '{SUBPASTA}' no banco {HOST}:{PORTA}?")

        if not seguir:
           raise ProcessoCanceladoException("Processo de resturação cancelado.")
        
        with pymysql.connect(host=HOST, user=USUARIO, password=PASSWORD, port=PORTA).cursor() as cursor:
            print(f"🟡 Restauração iniciada: {SUBPASTA}")
            bancos_sommusgestor = listar_databases_atuais(cursor)
            
            print("🔵 [1/3] - Deletando os bancos de dados")
            for banco in bancos_sommusgestor:
                deletar_database(cursor, banco)            

            arquivos_sql_backup = os.listdir(DIRETORIO_BACKUP)

            print("🔵 [2/3] - Criando os databases")
            for arquivo in arquivos_sql_backup:
                nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
                criar_database(cursor, nome_arquivo_sem_extensao)

            print("🔵 [3/3] - Restaurando os dados")
            for arquivo in arquivos_sql_backup:
                nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
                restaurar(nome_arquivo_sem_extensao, HOST, PORTA, USUARIO, PASSWORD, DIRETORIO_BACKUP)

        print(f"✅ Restauração finalizada em {datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}")

    except ProcessoCanceladoException as e:
        print(e)
    except Exception as e:
        print(f'⛔ [Exceção lançada] \n{e}')

 

🟡 Restauração iniciada: master_playlist_2023-12-29_08-58-49
🔵 [1/3] - Deletando os bancos de dados
🔵 [2/3] - Criando os databases
🔵 [3/3] - Restaurando os dados
   ⚪ Banco sommusgestor
   ⚪ Banco sommusgestor_1
   ⚪ Banco sommusgestor_2
   ⚪ Banco sommusgestor_8
   ⚪ Banco sommusgestor_9
   ⚪ Banco sommusgestor_hangfire
✅ Restauração finalizada em 29/12/2023 às 11:57:50
